<p align="center"><img src="../additionals/university-workshops-ytu-banner-en.png" alt="University Workshops" style="display: block; margin: 0 auto" height=/></p>

## Welcome to University Workshops!

Welcome to Empa Electronics' first University Workshop event, Yıldız Technical University Workshop!  

This code script will guide you to apply development steps of our Human Activity Recognition application.
  

**Steps in the Application:**

1. Requirements

2. Data Pre-processing

3. AI Model Definition

4. Model Exporting

5. Model Deployment

## 1. Requirements

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from math import floor
from scipy.stats import mode

from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

2024-05-20 12:33:25.885679: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 12:33:25.888078: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 12:33:25.921041: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path_dataset = "../datasets/Dataset_HAR_byEmpaElectronics.csv"

## 2. Data Pre-processing

### 2.1. Getting Raw Dataset

In [3]:
df_dataset = pd.read_csv(path_dataset)
df_dataset

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Labels
0,458,-890,-80,46,-44,-28,0
1,458,-888,-81,46,-44,-27,0
2,457,-888,-79,45,-44,-27,0
3,457,-888,-80,46,-45,-26,0
4,456,-887,-81,47,-45,-26,0
...,...,...,...,...,...,...,...
124923,251,-654,237,115,1269,-1115,2
124924,258,-599,200,121,1161,-1169,2
124925,237,-527,192,100,1000,-1202,2
124926,222,-503,180,1,886,-1224,2


### 2.2. Extracting Features & Labels

In [4]:
df_feats, df_labels = df_dataset.drop(columns=["Labels"]), df_dataset["Labels"]

Observing feature dataframe:

In [5]:
df_feats

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z
0,458,-890,-80,46,-44,-28
1,458,-888,-81,46,-44,-27
2,457,-888,-79,45,-44,-27
3,457,-888,-80,46,-45,-26
4,456,-887,-81,47,-45,-26
...,...,...,...,...,...,...
124923,251,-654,237,115,1269,-1115
124924,258,-599,200,121,1161,-1169
124925,237,-527,192,100,1000,-1202
124926,222,-503,180,1,886,-1224


Observing labels dataseries:

In [6]:
df_labels

0         0
1         0
2         0
3         0
4         0
         ..
124923    2
124924    2
124925    2
124926    2
124927    2
Name: Labels, Length: 124928, dtype: int64

Checking for null values features & labels:

In [7]:
print(f"[features] Number of NAs: {df_feats.isna().sum().sum()}")
print(f"[features] Number of nulls: {df_feats.isnull().sum().sum()}")
print(f"[labels] Number of NAs: {df_labels.isna().sum().sum()}")
print(f"[labels] Number of nulls: {df_labels.isnull().sum().item()}")

[features] Number of NAs: 0
[features] Number of nulls: 0
[labels] Number of NAs: 0
[labels] Number of nulls: 0


Getting classes as a list:

In [8]:
list_categories = np.array(sorted(set(df_labels.to_numpy().flatten()))).reshape(-1, 1)
list_categories

array([[0],
       [1],
       [2]])

### 2.3. Applying Label Encoding

Importing label one-hot encoder:

In [9]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

Fitting to your dataset classes:

In [10]:
encoder.fit(list_categories)

OneHotEncoder(sparse_output=False)

In [11]:
encoder.categories_

[array([0, 1, 2])]

Getting One-Hot encoded labels:

In [12]:
df_labels_ohe = pd.DataFrame(encoder.transform(df_labels.to_numpy().reshape(-1, 1)))
df_labels_ohe

,0,1,2
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
124923,0.0,0.0,1.0
124924,0.0,0.0,1.0
124925,0.0,0.0,1.0
124926,0.0,0.0,1.0


### 2.4. Create Sequence Batchs

Setting a sequence lenght and an overlapping ratio:

In [13]:
seq_length = 128
overlapping_ratio = 0.33

In [14]:
def create_sequences(data, labels, num_samples=104, overlap=0.5):
    """Takes tabular data and creates times sequences with given sample width."""

    # create empty lists for stacking
    data_sequences = []
    data_labels = []
    # get the number of examples
    num_examples = data.shape[0]
    # compute stride value
    strides = num_samples - floor(num_samples * overlap)
    # compute the number of sequences to use as iterator
    num_sequences = floor((num_examples - num_samples) / strides) + 1

    # iterate for sequence range
    for ind in range(num_sequences):

        # define start index
        ind_start = ind * strides
        # define end index
        ind_end = ind_start + num_samples
        # get the current data slice by using start and end indexes
        slice_seq_acc = data.values[ind_start:ind_end]
        # get the current labels slice by using start and end indexes
        slice_seq_label = labels.values[ind_start:ind_end]

        # take the modal value of label slice: (replace with .mode())
        label_seq = mode(slice_seq_label, keepdims=True)[0][0]

        # stack current slices
        data_sequences.append(slice_seq_acc)
        data_labels.append(label_seq)

    # convert stacks to numpy array
    data_sequences = np.array(data_sequences)
    data_labels = np.array(data_labels)

    # return sequence and label stacks as X and Y
    return data_sequences, data_labels

Creating sequences (windows) from our continuous data:

In [16]:
x_feats_seq, y_labels_seq = create_sequences(
                                    data= df_feats,
                                    labels= df_labels_ohe,
                                    num_samples= seq_length,
                                    overlap=overlapping_ratio)

In [17]:
print("Shape of Sequence Features:" , x_feats_seq.shape)
print("Shape of Sequence Labels:" , y_labels_seq.shape)

Shape of Sequence Features: (1452, 128, 6)
Shape of Sequence Labels: (1452, 3)


### 2.5. Splitting the Dataset as Train and Test Sets

Setting a train/test split ratio:

In [18]:
test_split_ratio = 0.3

Splitting the dataset:

In [19]:
x_train_seq, x_test_seq, y_train_seq, y_test_seq = train_test_split(x_feats_seq, y_labels_seq, test_size=test_split_ratio, shuffle=True)

In [20]:
print(f"Shapes of Train Set - Features: {x_train_seq.shape} - Labels: {y_train_seq.shape}")
print(f"Shapes of Test Set - Features: {x_test_seq.shape} - Labels: {y_test_seq.shape} ")

Shapes of Train Set - Features: (1016, 128, 6) - Labels: (1016, 3)
Shapes of Test Set - Features: (436, 128, 6) - Labels: (436, 3) 


## 3. Definition of AI Model

Defining a model creator function for our 1-D CNN model architecture:

In [21]:
def create_cnn_model(input_shape, output_shape):

        """Creates 1D-CNN model for sequence processing.

        Parameters:
            - input_shape: model input shape
            - output_shape: model output shape (number of classes)
        returns:
            - model: keras Sequential CNN model
        """

        model_cnn = Sequential(name="model_CNN")
        # Layer-1: Conv1D
        model_cnn.add(
            Conv1D(
                filters=64,
                kernel_size=3,
                activation="relu",
                padding="valid",
                strides=1,
                input_shape=input_shape,
            )
        )
        # Layer-2: Conv1D
        model_cnn.add(
            Conv1D(filters=32, kernel_size=3, activation="relu", padding="valid", strides=1)
        )
        # Layer-3: Dropout
        model_cnn.add(Dropout(0.4))
        # Layer-4: MaxPooling
        model_cnn.add(MaxPooling1D(pool_size=2, strides=2))
        # Layer-5: Flattening
        model_cnn.add(Flatten())
        # Layer-6: Fully-Connected
        model_cnn.add(Dense(units=32, activation="relu"))
        # Layer-Output: Softmax
        model_cnn.add(Dense(units=output_shape, activation="softmax"))
        # return CNN model object
        return model_cnn

And, defining a model trainer function that uses our model creator function:

In [22]:
def train_model(X, y, max_epochs=500, batch_size=128, lr=0.001, X_val=None, y_val=None):

    # get seq lenght and num of features
    seq_length, num_features = X.shape[1:]
    # get number of features and define input shape
    input_shape_cnn = seq_length, num_features
    # define output shape
    output_classes_cnn = y[0].size
    print("model input shape:", input_shape_cnn)
    print("output input shape:", output_classes_cnn)

    # create CNN model instance
    model = create_cnn_model(input_shape_cnn, output_classes_cnn)

    # compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["categorical_accuracy"],
    )
    # define EarlyStopping callback
    callback_early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="loss", min_delta=0, patience=5
    )
    # train a model for current param. combination
    history = model.fit(
        X,
        y,
        epochs=max_epochs,
        batch_size=batch_size,
        callbacks=[callback_early_stopping],
        validation_data=(X_val, y_val), # if  isinstance(None, (type(X_val), type(y_val))) else None,
    )

    # return training history and trained model
    return history, model

### 3.1. Starting Model Traning

In [21]:
history_cnn, model_cnn = train_model(x_train_seq, y_train_seq, X_val=x_test_seq, y_val=y_test_seq, max_epochs=100)

model input shape: (128, 6)
output input shape: 3


/home/denizcelik/VIRENVS/workshop_ytu_env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-20 12:27:31.761680: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - categorical_accuracy: 0.4807 - loss: 148.4674 - val_categorical_accuracy: 0.6858 - val_loss: 11.9276
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.7596 - loss: 15.7121 - val_categorical_accuracy: 0.8991 - val_loss: 4.6226
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.8734 - loss: 7.7375 - val_categorical_accuracy: 0.8899 - val_loss: 3.7028
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.9043 - loss: 3.9974 - val_categorical_accuracy: 0.9128 - val_loss: 2.5048
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - categorical_accuracy: 0.9092 - loss: 2.7559 - val_categorical_accuracy: 0.9220 - val_loss: 2.1264
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.9152 - loss: 2.5869 - val_categorical_accuracy: 0.9312 - val_loss: 1.6620
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - categorical_accuracy: 0.9195 - loss: 2.0784 

## 4. Model Exporting

### 4.1. Export as H5 File or Keras

In [29]:
model_cnn.save("../models/model_cnn_workshop_HAR_0_95_v5.h5")

In [23]:
model_cnn.save("../models/model_cnn_workshop_HAR_0_95_v5.keras")

### 4.2. Export as TFLite

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model_cnn)
tflite_model = converter.convert()

# Save the model.
with open('../models/model_cnn_workshop_HAR_0_95_v5.tflite', 'wb') as f:
  f.write(tflite_model)

### 4.3. Export as ONNX

In [24]:
import tf2onnx
import onnx
import onnxruntime as ort

In [25]:
x_val = np.ones((1, 128, 6), np.float32)
x_val.shape

(1, 128, 6)

In [ ]:
input_signature = [tf.TensorSpec([None, 128, 6], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model_cnn, input_signature, opset=13)

print("Keras result")
print(model_cnn(x_val).numpy())

print("ORT result")
sess = ort.InferenceSession(onnx_model.SerializeToString())
res = sess.run(None, {'x': x_val})
print(res[0])

In [ ]:
onnx.save(onnx_model, "../models/model_cnn_workshop_HAR_0_95_v5.onnx")

### 4.4. Loading & Testing Your Saved Model

In [27]:
import tensorflow as tf
model_loaded = tf.keras.models.load_model("../models/model_cnn_workshop_HAR_0_95_v5.keras")

/home/denizcelik/VIRENVS/workshop_ytu_env/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [28]:
model_loaded.summary()

Model: "model_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 126, 64)        │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 124, 32)        │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 124, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 62, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1984)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        63,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,024 (554.79 KB)

 Trainable params: 71,011 (277.39 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 71,013 (277.40 KB)

## 5. Model Deployment

_Continue to STM32 CubeAI_